# Q&A against a SQL Database

Now that we know (from the prior Notebook) how to query tabular data on a CSV file, let's try now to keep the data at is source and ask questions directly to a SQL Database.
The goal of this notebook is to demonstrate how a LLM so advanced as GPT-4 can understand a human question and translate that into a SQL query to get the answer. 

We will be using the Azure SQL Server that you created on the initial deployment. The server should be created on the Resource Group where the Azure Cognitive Search service is located.

Let's begin..

In [1]:
import os
import pandas as pd
import pyodbc
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain import SQLDatabaseChain

from app.prompts import MSSQL_PROMPT

# Don't mess with this unless you really know what you are doing
AZURE_OPENAI_API_VERSION = "2023-03-15-preview"

# Change these below with your own services credentials
AZURE_OPENAI_ENDPOINT = "Enter your Azure OpenAI Endpoint ..."
AZURE_OPENAI_API_KEY = "Enter your Azure OpenAI Key ..."

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["OPENAI_API_TYPE"] = "azure"

# Install MS SQL DB driver in your machine

We need the driver installed on this compute in order to talk to the SQL DB, so run the below cell once<br>
Reference: https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline

In [3]:
# !sudo ./download_odbc_driver.sh

# Load Azure SQL DB with the Covid Tracking CSV Data

The Azure SQL Database is currently empty, so we need to fill it up with data. Let's use the same data on the Covid CSV filed we used on the prior Notebook, that way we can compare results and methods. 
For this, you will need to type below the credentials you used at creation time.

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

# Define the parameters for the database connection
server = '<YOUR_SERVER_NAME>.database.windows.net'
database = '<YOUR_DATABASE_NAME>'
username = '<YOUR_DB_USERNAME>'
password = '<YOUR_DB_PASSWORD>'

# Don't change this driver variable
driver= '{ODBC Driver 18 for SQL Server}'

db_config = {
    'drivername': 'mssql+pyodbc',
    'username': username+'@'+server,
    'password': password,
    'host': server,
    'port': 1433,
    'database': database,
    'query': {'driver': 'ODBC Driver 18 for SQL Server'}
}

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Print the resulting URL string
# print(db_url)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection
try:
    conn = engine.connect()
    print("Connection successful!")
    result = engine.execute("SELECT @@Version")
    for row in result:
        print(row)
    conn.close()
    
except OperationalError:
    print("Connection failed.")

Connection successful!
('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tMar  8 2023 17:58:50 \n\tCopyright (C) 2022 Microsoft Corporation\n',)


/tmp/ipykernel_2430106/2265681346.py:36: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  result = engine.execute("SELECT @@Version")


In [5]:
# Read CSV file into a pandas dataframe
csv_path = "./data/all-states-history.csv"
df = pd.read_csv(csv_path).fillna(value = 0)

# Infer column names and data types
column_names = df.columns.tolist()
column_types = df.dtypes.to_dict()

# Generate SQL statement to create table
table_name = 'covidtracking'

create_table_sql = f"CREATE TABLE {table_name} ("
for name, dtype in column_types.items():
    if dtype == 'object':
        create_table_sql += f"{name} VARCHAR(255), "
    elif dtype == 'int64':
        create_table_sql += f"{name} INT, "
    elif dtype == 'float64':
        create_table_sql += f"{name} FLOAT, "
    elif dtype == 'bool':
        create_table_sql += f"{name} BIT, "
    elif dtype == 'datetime64[ns]':
        create_table_sql += f"{name} DATETIME, "
create_table_sql = create_table_sql[:-2] + ")"

try:
    engine.execute(create_table_sql)
except Exception as e:
    print(e)
    
# Insert data into SQL Database
try:
    df.to_sql(table_name, con=engine, if_exists='fail', index=False)
except Exception as e:
    print(e)

(pyodbc.ProgrammingError) ('42S01', "[42S01] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]There is already an object named 'covidtracking' in the database. (2714) (SQLExecDirectW)")
[SQL: CREATE TABLE covidtracking (date VARCHAR(255), state VARCHAR(255), death FLOAT, deathConfirmed FLOAT, deathIncrease INT, deathProbable FLOAT, hospitalized FLOAT, hospitalizedCumulative FLOAT, hospitalizedCurrently FLOAT, hospitalizedIncrease INT, inIcuCumulative FLOAT, inIcuCurrently FLOAT, negative FLOAT, negativeIncrease INT, negativeTestsAntibody FLOAT, negativeTestsPeopleAntibody FLOAT, negativeTestsViral FLOAT, onVentilatorCumulative FLOAT, onVentilatorCurrently FLOAT, positive FLOAT, positiveCasesViral FLOAT, positiveIncrease INT, positiveScore INT, positiveTestsAntibody FLOAT, positiveTestsAntigen FLOAT, positiveTestsPeopleAntibody FLOAT, positiveTestsPeopleAntigen FLOAT, positiveTestsViral FLOAT, recovered FLOAT, totalTestEncountersViral FLOAT, totalTestEncountersViralIncrease INT, to

# Query with LLM

In [6]:
# Create or LLM Langchain object using GPT-4 deployment
llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0, max_tokens=500)

In [7]:
# Let's use a type of Chain made for this type of SQL work.  
db = SQLDatabase.from_uri(db_url)
db_chain = SQLDatabaseChain(llm=llm, database=db, prompt=MSSQL_PROMPT, verbose=True)

In [8]:
# Let's check our prompt we created 
print(db_chain.prompt.template)


You are an MS SQL expert. Given an input question, first create a syntactically correct MS SQL query to run, then look at the results of the query and return the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the TOP clause as per MS SQL. You can order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in square brackets ([]) to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

## **Do not use double quotes on the SQL query**. 

** ALWAYS before giving the Final Answer, try another method**. Then reflect on the answers of the two methods you did and ask yours

In [17]:
# Natural Language question (query)
query_str = 'How may patients in total were hospitalized during July 2020 in Texas, and nationwide?'

In [18]:
db_chain(query_str)['result']



> Entering new SQLDatabaseChain chain...
How may patients in total were hospitalized during July 2020 in Texas, and nationwide?
SQLQuery:SELECT [state], SUM([hospitalizedIncrease]) as TotalHospitalized
FROM covidtracking
WHERE [date] >= '2020-07-01' AND [date] <= '2020-07-31'
GROUP BY [state]
ORDER BY [state]

SQLResult: [('AK', 83), ('AL', 7752), ('AR', 1439), ('AS', 0), ('AZ', 6524), ('CA', 0), ('CO', 952), ('CT', 539), ('DC', 0), ('DE', 0), ('FL', 12026), ('GA', 7638), ('GU', 0), ('HI', 66), ('IA', 0), ('ID', 513), ('IL', 0), ('IN', 1662), ('KS', 599), ('KY', 693), ('LA', 0), ('MA', 571), ('MD', 1749), ('ME', 40), ('MI', 0), ('MN', 1101), ('MO', 0), ('MP', 4), ('MS', 1120), ('MT', 129), ('NC', 0), ('ND', 133), ('NE', 280), ('NH', 128), ('NJ', 1718), ('NM', 784), ('NV', 0), ('NY', 0), ('OH', 2951), ('OK', 1641), ('OR', 562), ('PA', 0), ('PR', 0), ('RI', 210), ('SC', 2263), ('SD', 158), ('TN', 1996), ('TX', 0), ('UT', 933), ('VA', 1663), ('VI', 0), ('VT', 0), ('WA', 1245), ('WI', 11

'During July 2020, there were 0 patients hospitalized in Texas. Nationwide, the total number of patients hospitalized cannot be determined from the available data, as some states have 0 reported hospitalizations, which may indicate missing data.\n\nExplanation: I ran an SQL query to sum the hospitalizedIncrease column for each state during July 2020. The result for Texas was 0, which may indicate missing data. The nationwide total cannot be accurately determined due to the missing data in some states.'

### To use or not use Agents

As you can see above we achieved our goal of Question->SQL without using an Agent, we did it just by using a clever prompt. (If you want to see the different kind of prompts templates that come with langchain for sql chain, you can check it out [HERE](https://github.com/hwchase17/langchain/blob/master/langchain/chains/sql_database/prompt.py)). **So the question is, why do we need an Agent then?**

**This is why**: As we explained on the prior Notebook, an agent is a process in which the LLM self-asks about what approach and steps to take, questions the validity of the results and if sure, provides the answer. The SQLDatabaseChain doesn't do all this analysis, but instead tries a one-shot query in order to answer the question, which is good! but not enough for complex questions. That's why it couldn't solve the part about nationwide, it needs multiple steps in order to solve the problem, one query is not enough.
Notice that it did't pay atention to the prompt where we explicitly say to try two methods and reflect on the answer.

As homework, try to use an agent instead of a chain and get to the same result as Notebook 5

# Summary

In this notebook, we achieved our goal of Asking a Question in natural language to a dataset located on a SQL Database.  We did this by using purely prompt engineering (Langchain does it for us) and the cognitive power of GPT-4.

This process shows why it is NOT necessary to move the data from its original source as long as the source has an API and a common language we can use to interface with. GPT-4 has been trained on the whole public Github corpus, so it can pretty much understand most of the coding and database query languages that exists out there. 

In [20]:
import re

string = "There are many books in the library. A BOOK is found."
split_word = "book(s)?"
split_regex = re.compile(f"({split_word})", re.IGNORECASE)

result = split_regex.split(string)
print(result)

['There are many ', 'books', 's', ' in the library. A ', 'BOOK', None, ' is found.']


In [21]:
import re

string = "There are many books in the library. A BOOK is found."
split_word = "book"
split_regex = re.compile(f"{split_word}s?\\W*", re.IGNORECASE)

result = split_regex.split(string)
print(result)

['There are many ', 'in the library. A ', 'is found.']
